In [ ]:
import pandas as pd

import datetime as dt
from datetime import datetime
import time
import os

%run Util.ipynb

class DataSource(object):
    day_k = pd.DataFrame()
    week_k = pd.DataFrame()
    data = pd.DataFrame()
    
    # symbol=SZ#002335
    def __init__(self, symbol, level):
        self.symbol = symbol
        self.level = level
        
        self.data = pd.DataFrame()
        
        self.day_k = pd.DataFrame()
        self.week_k = pd.DataFrame()

    def merge_day_k_lines_to_one_week_k(self,day_k_lines):
        the_first_day = day_k_lines[:1]
        the_last_day = day_k_lines[-1:]

        w_date = the_last_day['date'].values[0]
        w_open = the_first_day['open'].values[0]
        w_high = day_k_lines['high'].max()
        w_low = day_k_lines['low'].min()
        w_close = the_last_day['close'].values[0]
        w_volume = day_k_lines['volume'].sum()
        w_amount = day_k_lines['amount'].sum()

        return [w_date,w_open,w_high,w_low,w_close,w_volume,w_amount]


    def get_day_k_in_that_week(self,cur_date):
        year = int(cur_date.split('-')[0])
        week = to_datetime(cur_date).isocalendar()[1]

        return self.day_k[(self.day_k['year']==year) & (self.day_k['week']==week)]


    def build_week_k_advance(self):
        file = day_dir+'D_'+self.symbol+'.txt'
        print(self.symbol+' file:'+file)
        stock_data = pd.read_csv(file,header=None, names=k_columns)

        period_type = 'W'
        stock_data['date'] = pd.to_datetime(stock_data['date'])

        stock_data.set_index('date',inplace=True)

        w_data = stock_data.resample(period_type).last()

        w_data['open'] = stock_data['open'].resample(period_type).first()
        w_data['high'] = stock_data['high'].resample(period_type).max()
        w_data['low'] = stock_data['low'].resample(period_type).min()
        w_data['volume'] = stock_data['volume'].resample(period_type).sum()
        w_data['amount'] = stock_data['amount'].resample(period_type).sum()

        w_data.reset_index(inplace=True)
        w_data['date'] = w_data['date'].apply(lambda x: x + dt.timedelta(days = -2))
        w_data['datetime'] = w_data['date']
        w_data['date'] = w_data['date'].apply(lambda x: x.strftime('%Y-%m-%d'))

        w_data = w_data[w_data['open'].notnull()]

        return w_data

    def get_day_K(self):
        file = day_dir+'D_'+self.symbol+'.txt'
        if(os.path.exists(file)):
            self.day_k = pd.read_csv(file, header=None, names=k_columns)

            self.day_k['datetime'] = pd.to_datetime(self.day_k['date'])
            self.day_k['year'] = self.day_k['date'].apply(lambda x: int(x.split('-')[0]))
            self.day_k['week'] = self.day_k['date'].apply(lambda x: to_datetime(x).isocalendar()[1])    
        else:
            print(self.symbol+ ' Warning! No day k lines')
        
        return self.day_k

    # symbol=SZ#002335
    def get_week_K(self):
        week_file = week_dir+ 'W_'+ self.symbol+'.txt'

        if(os.path.exists(week_file)):
            self.week_k =  pd.read_csv(week_file)
        else:
            print(self.symbol + ' No Week K ')
            self.week_k = self.build_week_k_file_by_day_K()

        self.week_k['datetime'] = pd.to_datetime(self.week_k['date'])
    #     self.week_k['year'] = week_k['date'].apply(lambda x: int(x.year))
    #     self.week_k['week'] = week_k['date'].apply(lambda x: int(x.week))    

        return self.week_k

    
    def get_data(self):
        if(self.level==level_day):
            self.data = self.get_day_K()
        elif(self.level==level_week):
            self.data = self.get_week_K()

        return self.data

    def build_week_k_file_by_day_K(self):
        week_k_df = self.build_week_k_advance()

        week_file = week_dir +'W_'+ self.symbol +'.txt'

        week_k_df.to_csv(week_file, index=0)

        return week_k_df
